In [3]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
import pickle
import sklearn.multiclass as sm
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 

In [4]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict
def normalize(x):
    min_val = np.min(x)
    max_val = np.max(x)
    x = (x-min_val) / (max_val-min_val)
    return x

def convert(data):
    im = data
    im_r = im[0:1024].reshape(32, 32).astype('float64')
    im_g = im[1024:2048].reshape(32, 32).astype('float64')
    im_b = im[2048:].reshape(32, 32).astype('float64')
    img = np.dstack((im_r, im_g, im_b))
#     grey = rgb2gray(img)
    return normalize(data).flatten()

In [5]:
def extract_data():
    train_data=[]
    for i in range(1,6):
        file='./cifar-10-batches-py/data_batch_'+str(i)
        temp=unpickle(file)
        temp_data=[]
        mat_data=[]
        for j in range(len(temp[b'data'])):
            op=convert(temp[b'data'][j])
            mat_data.append(op)
        temp_data.append(np.asarray(mat_data).astype('float64'))
        temp_data.append(np.asarray(temp[b'labels']).reshape(temp[b'data'].shape[0],1))
        train_data.append(temp_data)
    temp=unpickle('./cifar-10-batches-py/test_batch')
    temp_data=[]
    mat_data=[]
    for j in range(len(temp[b'data'])):
        op=convert(temp[b'data'][j])
        mat_data.append(op)
    temp_data.append(np.asarray(mat_data).astype('float64'))
    temp_data.append(np.asarray(temp[b'labels']).reshape(temp[b'data'].shape[0],1))
    test_data=temp_data
    temp=unpickle('./cifar-10-batches-py/batches.meta')
    labels=temp[b'label_names']
    for i in range(len(labels)):
        labels[i]=(labels[i].decode("utf-8"))
    return train_data,test_data,labels



In [38]:
def normalize123(data,test):
    data=data.astype('float64')
    test=test.astype('float64')
    mean = np.mean(data, axis = 0)
    data -= mean
    test -= mean
    return data,test

def roc(ovo,X_test,Y_test):
    probabilities = ovo.predict_proba(X_test)
    y_proba = probabilities[:, 1]
    false_positive_rate, true_positive_rate, thresholds = roc_curve(Y_test, y_proba, pos_label=1)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    plt.title('Receiver Operating Characteristic')
    roc_plot = plt.plot(false_positive_rate,
                        true_positive_rate,
                        label='AUC = {:0.2f}'.format(roc_auc))

    plt.legend(loc=0)
    plt.plot([0,1], [0,1], ls='--')
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')

def svm_linear(X,Y,x,y,model):
    X_train=X
    X_test=x
    Y_train=Y
    Y_test=y
    svm = SVC(kernel='linear')
    if model=='ovo':
        mod = sm.OneVsOneClassifier(svm)
    else:
        mod = sm.OneVsRestClassifier(svm)
    mod.fit(X_train,Y_train.ravel())
    y_pred = mod.predict(X_test)
    accuracy = accuracy_score(Y_test.ravel(), y_pred)
    print("Model accuracy is: ", accuracy)
    results = confusion_matrix(Y_test.ravel(),y_pred) 
    print("Confusion Matrix: ",results) 

def svm_rbf(X,Y,x,y,model):
    X_train=X
    X_test=x
    Y_train=Y
    Y_test=y
    svm = SVC(kernel='rbf')
    if model=='ovo':
        mod = sm.OneVsOneClassifier(svm)
    else:
        mod = sm.OneVsRestClassifier(svm)
    mod.fit(X_train,Y_train.ravel())
    y_pred = mod.predict(X_test)
    accuracy = accuracy_score(Y_test.ravel(), y_pred)
    print("Model accuracy is: ", accuracy)
    results = confusion_matrix(Y_test.ravel(),y_pred) 
    print("Confusion Matrix: ",results) 
    
def svm_poly(X,Y,x,y,model):
    X_train=X
    X_test=x
    Y_train=Y
    Y_test=y
    svm = SVC(kernel='poly')
    if model=='ovo':
        mod = sm.OneVsOneClassifier(svm)
    else:
        mod = sm.OneVsRestClassifier(svm)
    mod.fit(X_train,Y_train.ravel())
    y_pred = mod.predict(X_test)
    accuracy = accuracy_score(Y_test.ravel(), y_pred)
    print("Model accuracy is: ", accuracy)
    results = confusion_matrix(Y_test.ravel(),y_pred) 
    print("Confusion Matrix: ",results) 
#     roc(mod,X_test,Y_test.ravel())

In [7]:
def pca(train,test):
    pca = PCA(n_components=500, random_state=0, svd_solver='randomized')
    pca.fit(train[0])
    X_train = pca.transform(train[0])
    X_test = pca.transform(test[0])
    return X_train,X_test
    

In [8]:
def pca_proc(i,test_data):
    train,test=pca(i,test_data)
    train=[train,i[1]]
    test=[test,test_data[1]]
    return [train,test]
   

In [26]:
def split_train(data):
    split=[]
    split.append([data[0][:2000],data[1][:2000]])
    split.append([data[0][2000:4000],data[1][2000:4000]])
    split.append([data[0][4000:6000],data[1][4000:6000]])
    split.append([data[0][6000:8000],data[1][6000:8000]])
    split.append([data[0][8000:],data[1][8000:]])
    return split

In [9]:
train_data,test_data,labels=extract_data()
for i in train_data:
    temp_test=test_data
    i[0],temp_test[0]=normalize123(i[0],temp_test[0])


(10000, 3072)


In [32]:
k_cross_split=split_train(train_data[0])
trainx=[]
trainy=[]

trainx.append(np.vstack((k_cross_split[1][0],k_cross_split[2][0],k_cross_split[3][0],k_cross_split[4][0])))
trainy.append(np.vstack((k_cross_split[1][1],k_cross_split[2][1],k_cross_split[3][1],k_cross_split[4][1])))

trainx.append(np.vstack((k_cross_split[0][0],k_cross_split[2][0],k_cross_split[3][0],k_cross_split[4][0])))
trainy.append(np.vstack((k_cross_split[0][1],k_cross_split[2][1],k_cross_split[3][1],k_cross_split[4][1])))

trainx.append(np.vstack((k_cross_split[1][0],k_cross_split[0][0],k_cross_split[3][0],k_cross_split[4][0])))
trainy.append(np.vstack((k_cross_split[1][1],k_cross_split[0][1],k_cross_split[3][1],k_cross_split[4][1])))

trainx.append(np.vstack((k_cross_split[1][0],k_cross_split[2][0],k_cross_split[0][0],k_cross_split[4][0])))
trainy.append(np.vstack((k_cross_split[1][1],k_cross_split[2][1],k_cross_split[0][1],k_cross_split[4][1])))

trainx.append(np.vstack((k_cross_split[1][0],k_cross_split[2][0],k_cross_split[3][0],k_cross_split[0][0])))
trainy.append(np.vstack((k_cross_split[1][1],k_cross_split[2][1],k_cross_split[3][1],k_cross_split[0][1])))

testx=[k_cross_split[0][0],k_cross_split[1][0],k_cross_split[2][0],k_cross_split[3][0],k_cross_split[4][0]]
testy=[k_cross_split[0][1],k_cross_split[1][1],k_cross_split[2][1],k_cross_split[3][1],k_cross_split[4][1]]


In [40]:
def call_svm(X,Y,x,y):
    print("linear svm ovo start")
    svm_linear(X,Y,x,y,'ovo')
    print("linear svm ovr start")
    svm_linear(X,Y,x,y,'ovr')
#     svm_rbf(X,Y,x,y,'ovo')
#     svm_rbf(X,Y,x,y,'ovr')
#     svm_poly(X,Y,x,y,'ovo')
#     svm_poly(X,Y,x,y,'ovr')

In [ ]:
for i in range(5):
    print("svm for "+str(i+1)+" started")
    call_svm(trainx[i],trainy[i],testx[i],testy[i])
    break

svm for 1 started
